In [1]:
from sklearn import datasets
from pybrain.datasets.classification import ClassificationDataSet
from pybrain.tools.shortcuts import buildNetwork
from pybrain.supervised.trainers import BackpropTrainer

In [3]:
iris = datasets.load_iris()
x, y = iris.data, iris.target

dataset = ClassificationDataSet(4, 1, nb_classes=3)
for i in range(len(x)):
    dataset.addSample(x[i], y[i])


train_data_temp, part_data_temp = dataset.splitWithProportion(0.6)
test_data_temp, val_data_temp = part_data_temp.splitWithProportion(0.5)

train_data = ClassificationDataSet(4, 1, nb_classes=3)
for n in range(train_data_temp.getLength()):
    train_data.addSample(train_data_temp.getSample(n)[0], train_data_temp.getSample(n)[1])

test_data = ClassificationDataSet(4, 1, nb_classes=3)
for n in range(test_data_temp.getLength()):
    test_data.addSample(test_data_temp.getSample(n)[0], test_data_temp.getSample(n)[1])

val_data = ClassificationDataSet(4, 1, nb_classes=3)
for n in range(val_data_temp.getLength()):
    val_data.addSample(val_data_temp.getSample(n)[0], val_data_temp.getSample(n)[1])


print(len(train_data))
print(len(test_data))
print(len(val_data))

90
30
30


In [4]:
print('Antes do _convertToOneOfMany')
print(train_data['target'][:5])

train_data._convertToOneOfMany()
test_data._convertToOneOfMany()
val_data._convertToOneOfMany()

print('Depois do _convertToOneOfMany')
print(train_data['target'][:5])

print('----------------------------------')

Antes do _convertToOneOfMany
[[1]
 [2]
 [2]
 [1]
 [1]]
Depois do _convertToOneOfMany
[[0 1 0]
 [0 0 1]
 [0 0 1]
 [0 1 0]
 [0 1 0]]


In [19]:
from pybrain.structure.modules import SoftmaxLayer

net = buildNetwork(train_data.indim, 5, train_data.outdim, outclass=SoftmaxLayer)
trainer = BackpropTrainer(net, dataset=train_data, learningrate=0.01, momentum=0.1)
trainer.trainOnDataset(train_data, 500)

In [20]:
from pybrain.utilities import percentError

out_test = net.activateOnDataset(test_data).argmax(axis=1)
print('Erro de teste: {}'.format(percentError(out_test, test_data['class'])))

out_val = net.activateOnDataset(val_data).argmax(axis=1)
print('Erro de validação: {}'.format(percentError(out_val, val_data['class'])))

print('-------------------------------------------------------------')

print('Teste')
print('Saída da rede:\t', out_test)
print('correto:\t', test_data['class'][:, 0])

print('-------------------------------------------------------------')

print('Validação')
print('Saída da rede:\t', out_val)
print('correto:\t', val_data['class'][:, 0])

Erro de teste: 6.666666666666667
Erro de validação: 3.3333333333333335
-------------------------------------------------------------
Teste
Saída da rede:	 [0 2 1 1 1 2 0 2 2 0 2 1 2 1 1 2 2 0 0 0 1 0 2 1 1 0 1 1 1 1]
correto:	 [0 2 1 1 1 2 0 1 2 0 2 1 2 1 1 1 2 0 0 0 1 0 2 1 1 0 1 1 1 1]
-------------------------------------------------------------
Validação
Saída da rede:	 [0 0 2 2 0 1 2 1 2 2 2 1 2 0 2 2 2 1 0 0 0 2 2 2 0 1 0 2 0 0]
correto:	 [0 0 2 1 0 1 2 1 2 2 2 1 2 0 2 2 2 1 0 0 0 2 2 2 0 1 0 2 0 0]


In [7]:
import numpy as np

out_val = net.activateOnDataset(val_data).argmax(axis=1)
print('Erro de validação: {}'.format(percentError(out_val, val_data['class'])))

Erro de validação: 0.0
